# KNN - Defunciones Fetales

En este Jupyter Notebook, se busca crear un modelo de Defunciones Fetales para poder predecir el tipo de defunción fetal que se puede presentar en un embarazo, con base en las características de la madre y del embarazo.

----------
### 1. Importar librerías

In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from mlxtend.frequent_patterns import apriori
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_classif
from prettytable import PrettyTable
import itertools
from sklearn.inspection import permutation_importance
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

-----------
### 2. Cargar datos
Como primer paso, se cargan los datos ya explorados y limpios. Estos datos se encuentran en el archivo "defunciones_clean.csv".

Las variables presentes son:

    'Área_Geográfica',
    'Asistencia_Recibida',
    'Año_Ocurrencia',
    'Año_Registro',
    'Causa_Defunción',
    'Clase_Parto',
    'Departamento_Ocurrencia',
    'Departamento_Registro',
    'Día_Ocurrencia',
    'Edad_Madre',
    'Estado_Civil_Madre',
    'Escolaridad_Madre',
    'Mes_Ocurrencia',
    'Mes_Registro',
    'Municipio_Ocurrencia',
    'Municipio_Registro',
    'Municipio_Residencia_Madre',
    'Ocupación_Madre',
    'Semanas_Gestacion',
    'Sexo',
    'Sitio_Ocurrencia',
    'Tipo_Atencion',
    'Total_Hijos_Nacidos',
    'Total_Hijos_Nacidos_Muertos',
    'Total_Hijos_Vivos',
    'Via_Parto',
    'Nacionalidad_Madre',
    'Grupo_Etnico_Madre',

En este caso, se determinó que las clases no estaban balanceadas. Por tal razón, se optó por un balanceo de datos a través de over sampling. Para esto, se utilizó la funcion RandomOverSampler.

In [76]:
# Cargar datos
defunciones = pd.read_csv('defunciones_clean.csv').round(0)
defunciones = defunciones[defunciones.Causa_Defunción != 99]

X = defunciones.drop(['Causa_Defunción'], axis=1)
y = defunciones['Causa_Defunción']
oversample = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversample.fit_resample(X, y)

defunciones = pd.concat([X_over, y_over], axis=1)

defunciones['Causa_Defunción'].value_counts()


0    35271
1    35271
Name: Causa_Defunción, dtype: int64

In [77]:
# Eliminar columnas no relevantes (segun analisis exploratorio anteriormente realizado)
defunciones.drop('Departamento_Registro', axis=1, inplace=True)
defunciones.drop('Municipio_Registro', axis=1, inplace=True)

defunciones.drop('Municipio_Ocurrencia', axis=1, inplace=True)   
defunciones.drop('Departamento_Residencia_Madre', axis=1, inplace=True) 

defunciones.drop('Municipio_Residencia_Madre', axis=1, inplace=True)
defunciones.drop('Departamento_Ocurrencia', axis=1, inplace=True)

defunciones.drop('Año_Ocurrencia', axis=1, inplace=True)
defunciones.drop('Mes_Ocurrencia', axis=1, inplace=True)
defunciones.drop('Día_Ocurrencia', axis=1, inplace=True)        # No se considera el día de ocurrencia

defunciones.drop('Año_Registro', axis=1, inplace=True)
defunciones.drop('Mes_Registro', axis=1, inplace=True)

defunciones.head()

,Área_Geográfica,Asistencia_Recibida,Clase_Parto,Edad_Madre,Estado_Civil_Madre,Escolaridad_Madre,Ocupación_Madre,Semanas_Gestacion,Sexo,Sitio_Ocurrencia,Tipo_Atencion,Total_Hijos_Nacidos,Total_Hijos_Nacidos_Muertos,Total_Hijos_Vivos,Via_Parto,Nacionalidad_Madre,Grupo_Etnico_Madre,Causa_Defunción
0,99,6,1,18.0,2,99,99,34.0,1,2,1,1.0,2.0,1.0,99,1,4,0
1,99,1,1,36.0,2,99,1,34.0,1,9,1,1.0,3.0,2.0,99,1,4,0
2,99,2,1,22.0,2,99,99,34.0,1,2,1,1.0,3.0,2.0,99,1,4,0
3,99,2,1,26.0,2,99,99,34.0,1,2,1,1.0,0.0,0.0,99,1,4,0
4,99,2,1,43.0,2,99,99,34.0,2,2,1,1.0,10.0,6.0,99,1,4,0


----------
### 3. Selección de variables a utilizar en modelo

En este caso, se busca determinar (a través de SelectKBest y Chi2) las mejores variables para utilizar en el modelo a realizar. Especificamente, se optó por utilizar las 20 variables que tienen una mayor correlación con la variable objetivo (Causa_Defunción).

El código mostrado a continuación selecciona los mejores features del conjunto de datos utilizando la prueba chi-cuadrado (chi-squared test). En concreto, se utiliza la función "SelectKBest" para seleccionar el número "k" de features que se especifican (en este caso, solo el mejor feature) que tienen una mayor relación con la variable objetivo. 

> En otras palabras, se analiza la correlación entre las variables y la variable objetivo, y se seleccionan las variables que tienen una mayor correlación.

In [78]:
# Split the dataset into features (X) and target variable (y)
X = defunciones.drop('Causa_Defunción', axis=1)
y = defunciones['Causa_Defunción']

# Apply feature selection using the chi-squared test
selector = SelectKBest(chi2, k=1)
X_new = selector.fit_transform(X, y)

# Order X_new by feature importance
order = np.argsort(selector.scores_)[::-1]

# Create a list of sorted feature names
feature_names = []
for i in order:
    feature_names.append(X.columns[i])

    if len(feature_names) == 20:
        break

print(feature_names)

X_new = X[feature_names]

['Área_Geográfica', 'Via_Parto', 'Sitio_Ocurrencia', 'Escolaridad_Madre', 'Nacionalidad_Madre', 'Estado_Civil_Madre', 'Tipo_Atencion', 'Clase_Parto', 'Grupo_Etnico_Madre', 'Sexo', 'Ocupación_Madre', 'Asistencia_Recibida', 'Semanas_Gestacion', 'Edad_Madre', 'Total_Hijos_Nacidos_Muertos', 'Total_Hijos_Nacidos', 'Total_Hijos_Vivos']


----------
### 4. Creación del modelo KNN

A continuación se genera el modelo KNN con las variables seleccionadas que tienen mayor correlación con la variable objetivo. Asimismo, se presenta el accuracy del modelo; el cual es 0.93. Esto indica que el modelo es capaz de predecir 89 de 100 veces correctamente el tipo de muerte fetal.

In [79]:
# Separar en train, test y validation
X_new = pd.DataFrame(X_new)
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

print(y_train.value_counts())
print(y_test.value_counts())
print(y_val.value_counts())

0    24723
1    24656
Name: Causa_Defunción, dtype: int64
0    5373
1    5208
Name: Causa_Defunción, dtype: int64
1    5407
0    5175
Name: Causa_Defunción, dtype: int64


In [80]:
# GridSearchCV for the best number of neighbors
grid_params = {'n_neighbors': [10, 25, 50, 100, 150, 200, 250]}

knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn, grid_params, cv=5)
knn_cv.fit(X_val, y_val)

print(knn_cv.best_params_)

{'n_neighbors': 10}


In [81]:
# Create the KNN model
knn = KNeighborsClassifier(n_neighbors=10)

# Fit the model to the training data
knn.fit(X_train, y_train)

# Predict on the test set
y_pred = knn.predict(X_test)

# Precision del Modelo
print("Precision:",precision_score(y_test, y_pred, average='weighted'))

# Recall del Modelo
print("Recall:",recall_score(y_test, y_pred, average='weighted'))

# F1 Score del Modelo
print("F1 Score",f1_score(y_test, y_pred, average='weighted'))

# Accuracy del Modelo
print("Accuracy:", knn.score(X_test, y_test))


# Matriz de Confusion
confusion_matrix(y_test, y_pred)

Precision: 0.938008253208971
Recall: 0.9307248842264436
F1 Score 0.9305099030000438
Accuracy: 0.9307248842264436


array([[4668,  705],
       [  28, 5180]])

----------
### 5. Proceso Investigativo

En base al modelo generado, se pueden ir alterando ciertos datos para ver cómo se comporta el modelo. Por ejemplo, se puede cambiar la edad de la madre, la escolaridad, el estado civil, etc. y ver cómo se comporta el modelo. Esto con el fin de poder determinar qué características son las que más afectan el resultado de la predicción.

Según las recomendaciones del experto (Dr. González), se analizó como las siguientes variables afectan la probabilidad de las causas de defunción fetal.

#### 5.1 Escolaridad Madre

In [82]:
# Cual es el nivel de educacion de las madres con Causa_Defunción = 0 = Parto
defunciones_0 = defunciones[defunciones['Causa_Defunción'] == 0]
defunciones_0['Escolaridad_Madre'].value_counts()

1     10999
2     10369
99     6113
3      3991
4      3303
5       493
6         3
Name: Escolaridad_Madre, dtype: int64

In [83]:
# Cual es el nivel de educacion de las madres con Causa_Defunción = 1 = Embarazo
defunciones_1 = defunciones[defunciones['Causa_Defunción'] == 1]
defunciones_1['Escolaridad_Madre'].value_counts()

2     13411
1      9949
3      5077
4      3058
99     2806
5       970
Name: Escolaridad_Madre, dtype: int64

In [84]:
# Generar nuevo dataset donde Causa_Defunción = 1 = Embarazo

defunciones_1 = defunciones[defunciones['Causa_Defunción'] == 1]
defunciones_1 = defunciones_1[feature_names+['Causa_Defunción']]

defunciones_1['Escolaridad_Madre'] = 5  #Universitaria 

# Separar datos prueba y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(defunciones_1.drop('Causa_Defunción', axis=1), defunciones_1['Causa_Defunción'], test_size=0.3, random_state=42)

# Pasar prueba a modelo KNN existente
y_pred = knn.predict(X_test)

# Porcentaje de cambio en la predicción
print("ESCOLARIDAD: Universidad - Embarazo")
print("% Cambio", (y_pred != y_test).sum() / len(y_test)*100, "%")

ESCOLARIDAD: Universidad - Embarazo
% Cambio 43.09204309204309 %


In [85]:
# Generar nuevo dataset donde Causa_Defunción = 1 = Embarazo

defunciones_1 = defunciones[defunciones['Causa_Defunción'] == 1]
defunciones_1 = defunciones_1[feature_names+['Causa_Defunción']]

defunciones_1['Escolaridad_Madre'] = 6  #Postgrado 

# Separar datos prueba y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(defunciones_1.drop('Causa_Defunción', axis=1), defunciones_1['Causa_Defunción'], test_size=0.3, random_state=42)

# Pasar prueba a modelo KNN existente
y_pred = knn.predict(X_test)

# Porcentaje de cambio en la predicción
print("ESCOLARIDAD: Postgrado - Embarazo")
print("% Cambio", (y_pred != y_test).sum() / len(y_test)*100, "%")

ESCOLARIDAD: Postgrado - Embarazo
% Cambio 53.24135324135324 %


#### 5.2 Via de Parto

In [86]:
# Cual es tipo de parto mas comun con Causa_Defunción = 0 = Parto
defunciones_0 = defunciones[defunciones['Causa_Defunción'] == 0]
defunciones_0['Via_Parto'].value_counts()

1     24625
2      7450
99     3196
Name: Via_Parto, dtype: int64

In [87]:
# Cual es tipo de parto mas comun con Causa_Defunción = 1 = Embarazo
defunciones_1 = defunciones[defunciones['Causa_Defunción'] == 1]
defunciones_1['Via_Parto'].value_counts()

1     24376
2     10025
99      870
Name: Via_Parto, dtype: int64

In [88]:
# Generar nuevo dataset donde Causa_Defunción = 1 = Embarazo

defunciones_1 = defunciones[defunciones['Causa_Defunción'] == 1]
defunciones_1 = defunciones_1[feature_names+['Causa_Defunción']]

defunciones_1['Via_Parto'] = 1  #Vaginal 

# Separar datos prueba y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(defunciones_1.drop('Causa_Defunción', axis=1), defunciones_1['Causa_Defunción'], test_size=0.3, random_state=42)

# Pasar prueba a modelo KNN existente
y_pred = knn.predict(X_test)

# Porcentaje de cambio en la predicción
print("VIA PARTO: Vaginal - Embarazo")
print("% Cambio", (y_pred != y_test).sum() / len(y_test)*100, "%")

VIA PARTO: Vaginal - Embarazo
% Cambio 0.9733509733509733 %


In [89]:
# Generar nuevo dataset donde Causa_Defunción = 1 = Embarazo

defunciones_1 = defunciones[defunciones['Causa_Defunción'] == 1]
defunciones_1 = defunciones_1[feature_names+['Causa_Defunción']]

defunciones_1['Via_Parto'] = 2  #Cesarea 

# Separar datos prueba y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(defunciones_1.drop('Causa_Defunción', axis=1), defunciones_1['Causa_Defunción'], test_size=0.3, random_state=42)

# Pasar prueba a modelo KNN existente
y_pred = knn.predict(X_test)

# Porcentaje de cambio en la predicción
print("VIA PARTO: Cesarea - Embarazo")
print("% Cambio", (y_pred != y_test).sum() / len(y_test)*100, "%")

VIA PARTO: Cesarea - Embarazo
% Cambio 0.7938007938007937 %


#### 5.3 Area Geografica

In [90]:
# Cual es el area geografica mas comun con Causa_Defunción = 0 = Parto
defunciones_0 = defunciones[defunciones['Causa_Defunción'] == 0]
defunciones_0['Área_Geográfica'].value_counts()

1     27744
2      4526
99     3001
Name: Área_Geográfica, dtype: int64

In [91]:
# Cual es el area geografica mas comun con Causa_Defunción = 1 = Embarazo
defunciones_1 = defunciones[defunciones['Causa_Defunción'] == 1]
defunciones_1['Área_Geográfica'].value_counts()

1     30733
2      3873
99      665
Name: Área_Geográfica, dtype: int64

In [92]:
# Generar nuevo dataset donde Causa_Defunción = 0 = Parto

defunciones_0 = defunciones[defunciones['Causa_Defunción'] == 0]
defunciones_0 = defunciones_0[feature_names+['Causa_Defunción']]

defunciones_0['Área_Geográfica'] = 1  #Urbano 

# Separar datos prueba y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(defunciones_0.drop('Causa_Defunción', axis=1), defunciones_0['Causa_Defunción'], test_size=0.3, random_state=42)

# Pasar prueba a modelo KNN existente
y_pred = knn.predict(X_test)

# Porcentaje de cambio en la predicción
print("AREA GEOGRAFICA: Urbano - Parto")
print("% Cambio", (y_pred != y_test).sum() / len(y_test)*100, "%")

AREA GEOGRAFICA: Urbano - Parto
% Cambio 11.566811566811566 %


##### 5.4 Asistencia Recibida

In [93]:
# Cual es la asistencia recibida mas comun con Causa_Defunción = 0 = Parto
defunciones_0 = defunciones[defunciones['Causa_Defunción'] == 0]
defunciones_0['Asistencia_Recibida'].value_counts()

1    24180
2     5362
3     3123
4     2139
5      246
6      221
Name: Asistencia_Recibida, dtype: int64

In [94]:
# Cual es la asistencia recibida mas comun con Causa_Defunción = 1 = Embarazo
defunciones_1 = defunciones[defunciones['Causa_Defunción'] == 1]
defunciones_1['Asistencia_Recibida'].value_counts()

1    27804
2     4512
3     1958
4      629
5      313
6       55
Name: Asistencia_Recibida, dtype: int64

In [95]:
# Generar nuevo dataset donde Causa_Defunción = 0 = Parto

defunciones_0 = defunciones[defunciones['Causa_Defunción'] == 0]
defunciones_0 = defunciones_0[feature_names+['Causa_Defunción']]

defunciones_0['Asistencia_Recibida'] = 1  #Medico 

# Separar datos prueba y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(defunciones_0.drop('Causa_Defunción', axis=1), defunciones_0['Causa_Defunción'], test_size=0.3, random_state=42)

# Pasar prueba a modelo KNN existente
y_pred = knn.predict(X_test)

# Porcentaje de cambio en la predicción
print("ASISTENCIA RECIBIDA: Medica - Embarazo")
print("% Cambio (Muerte Embarazo -> Muerte Parto)", (y_pred != y_test).sum() / len(y_test)*100, "%")

ASISTENCIA RECIBIDA: Medica - Embarazo
% Cambio (Muerte Embarazo -> Muerte Parto) 11.718011718011718 %
